In [1]:
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install -U scikit-learn

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl (12.8 MB)


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from collections import defaultdict 
import spacy
import en_core_web_sm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression # just to see if the model is even working
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from collections import defaultdict
from plotly import express as px
from sklearn.metrics import accuracy_score
import re
from collections import Counter 

In [2]:
import os
print(os.getcwd())

/Users/sebastian/Documents/code-things/DS3-Ted-Talk/DS3-Ted-Talk


In [3]:
raw = pd.read_csv('ted_talks_en.csv')

In [4]:
raw.head()

,talk_id,title,speaker_1,all_speakers,occupations,about_speakers,views,recorded_date,published_date,event,native_lang,available_lang,comments,duration,topics,related_talks,url,description,transcript
0,1,Averting the climate crisis,Al Gore,{0: 'Al Gore'},{0: ['climate advocate']},{0: 'Nobel Laureate Al Gore focused the world’...,3523392,2006-02-25,2006-06-27,TED2006,en,"['ar', 'bg', 'cs', 'de', 'el', 'en', 'es', 'fa...",272.0,977,"['alternative energy', 'cars', 'climate change...","{243: 'New thinking on the climate crisis', 54...",https://www.ted.com/talks/al_gore_averting_the...,With the same humor and humanity he exuded in ...,"Thank you so much, Chris. And it's truly a gre..."
1,92,The best stats you've ever seen,Hans Rosling,{0: 'Hans Rosling'},{0: ['global health expert; data visionary']},"{0: 'In Hans Rosling’s hands, data sings. Glob...",14501685,2006-02-22,2006-06-27,TED2006,en,"['ar', 'az', 'bg', 'bn', 'bs', 'cs', 'da', 'de...",628.0,1190,"['Africa', 'Asia', 'Google', 'demo', 'economic...","{2056: ""Own your body's data"", 2296: 'A visual...",https://www.ted.com/talks/hans_rosling_the_bes...,You've never seen data presented like this. Wi...,"About 10 years ago, I took on the task to teac..."
2,7,Simplicity sells,David Pogue,{0: 'David Pogue'},{0: ['technology columnist']},{0: 'David Pogue is the personal technology co...,1920832,2006-02-24,2006-06-27,TED2006,en,"['ar', 'bg', 'de', 'el', 'en', 'es', 'fa', 'fr...",124.0,1286,"['computers', 'entertainment', 'interface desi...","{1725: '10 top time-saving tech tips', 2274: '...",https://www.ted.com/talks/david_pogue_simplici...,New York Times columnist David Pogue takes aim...,"(Music: ""The Sound of Silence,"" Simon & Garfun..."
3,53,Greening the ghetto,Majora Carter,{0: 'Majora Carter'},{0: ['activist for environmental justice']},{0: 'Majora Carter redefined the field of envi...,2664069,2006-02-26,2006-06-27,TED2006,en,"['ar', 'bg', 'bn', 'ca', 'cs', 'de', 'en', 'es...",219.0,1116,"['MacArthur grant', 'activism', 'business', 'c...",{1041: '3 stories of local eco-entrepreneurshi...,https://www.ted.com/talks/majora_carter_greeni...,"In an emotionally charged talk, MacArthur-winn...",If you're here today — and I'm very happy that...
4,66,Do schools kill creativity?,Sir Ken Robinson,{0: 'Sir Ken Robinson'},"{0: ['author', 'educator']}","{0: ""Creativity expert Sir Ken Robinson challe...",65051954,2006-02-25,2006-06-27,TED2006,en,"['af', 'ar', 'az', 'be', 'bg', 'bn', 'ca', 'cs...",4931.0,1164,"['children', 'creativity', 'culture', 'dance',...","{865: 'Bring on the learning revolution!', 173...",https://www.ted.com/talks/sir_ken_robinson_do_...,Sir Ken Robinson makes an entertaining and pro...,Good morning. How are you? (Audience) Good. It...


In [5]:
arr = raw['topics'].to_numpy()
keywords = defaultdict(int)
for i, inner in enumerate(arr):
    x = inner.split("'")
    for i, tag in enumerate(x):
        if i % 2 != 0 and i != 0 and i != len(x) - 1:
            keywords[tag.lower()] += 1
len(keywords.keys())

458

In [6]:
most_common_keywords = sorted(list(keywords.keys()), key=lambda keyword: keywords[keyword])[::-1]
occurences = [keywords[keyword] for keyword in most_common_keywords]
#list(zip(most_common_keywords, occurences))

In [7]:
fig = px.histogram(pd.DataFrame({"Number of Speeches per Label": occurences}),
                    title="Unique Topic Frequency",
                    x="Number of Speeches per Label")
fig.add_vline(x=50, line_color="red")
fig.add_annotation(x=700, y=200, text="Unique Topics Kept")
fig.add_annotation(x=25, y=200, text="Unique Topics Discarded")
fig.show()

In [8]:
cutoff = 50
y = filter(lambda x: x >= cutoff, occurences)
print(f"Amount of topics with occurences >= {cutoff} : {len(list(y))}")

Amount of topics with occurences >= 50 : 156


In [9]:
def convert_topics(topic):
    x = topic.split("'")
    tags = []
    for i, tag in enumerate(x):
        if i % 2 != 0 and i != 0 and i != len(x) - 1:
            tags.append(tag.lower())
    return tags

def correct_dataframe_topics_column(df, in_place=False):
    if not in_place:
        df = df.copy()
    topics_listed_column = df['topics'].apply(convert_topics)
    df['topics'] = topics_listed_column
    return df

In [10]:
"""
Topics of focus:
STEM - Science, Technology, Engineering, Math(Mathematics), Psychology, Physics, Biotech, AI, Cognitive Science
Culture - Culture, Society, Community
Business - Business, Economics
Social Change - social change/activism
History - History/Politics
"""
topic_to_group = defaultdict(lambda: 'not-relevant')
stem = ["science", "technology", "engineering", "math", "mathematics", "psychology", "physics", "biotech", "ai", "cognitive science", 'math', 'computers', 'science', 'software', 'technology', 'visualizations', 'health']
culture = ["culture", "society", "community", 'culture', 'humor', 'visualizations', 'children', 'creativity', 'entertainment', 'media', 'education', 'parenting', 'teaching', 'collaboration', 'music', 'performance', 'activism', 'africa', 'inequality', 'politics']
business = ["business", "economics",  'design', 'comedy', 'storytelling', 'entertainment', 'media', 'business', 'cities', 'demo', 'economics']
activism = ['social change', 'activism',  'climate change', 'environment', 'global issues', 'sustainability', 'green', 'inequality', 'global development', 'pollution']
history = ['history', 'politics',  'creativity', 'architecture', 'religion', 'storytelling']
categories = [ ['stem'] + stem, ['culture'] + culture, ['business'] + business,
              ['activism'] + activism, ['history'] + history]

for category in categories:
    cat = None
    for i, topic in enumerate(category):
        if i == 0:
            cat = topic
            continue
        topic_to_group[topic] = cat

In [11]:
def group_topics(topic_list):
    grouped = defaultdict(int)
    for topic in topic_list:
        grouped[topic_to_group[topic]] += 1
    vals_with_keys = sorted(
        [(topic, grouped[topic]) for topic in grouped.keys()],
        key=lambda tup: tup[1])[::-1]
    for tup in vals_with_keys: 
        if tup[0] != 'not-relevant': 
            return tup[0]
    return "not-relevant"

def group_df_topics(df, in_place=False):
    if not in_place:
        df = df.copy()
    df['grouped_topics'] = df['topics'].apply(group_topics)
    return df

In [12]:
def count_unique_words(script): 
    unique_words = dict()
    script_lst = script.split()
    for word in script_lst: 
        if word in unique_words: 
            unique_words[word] += 1
        else: 
            unique_words[word] = 1
    return unique_words

def add_words(df, in_place=False):
    if in_place:
        df = df.copy()
    df["clean_scripts"] = (
    df["transcript"]
    .str.replace(r'\(Music: [^)]+\) ','', regex = True)
    .str.replace(r' — ', ' ', regex = True)
    .str.replace(r'[[."\'!\],:?;=+*&]', '', regex = True)
    .str.lower())
    df["words"] = df["clean_scripts"].apply(count_unique_words)
    return df

In [13]:
# NOT USING RIGHT NOW 
def one_hot_encode(df: pd.DataFrame, verbose=False, in_place=False):
    """
    expects df to have a columns ['words'] which contains a dictionary for each entry,
    this dictionary has all the unique words in this row's transcript as keys, and their values
    correspond the amount of time that word appears in the transcript. This creates 
    """
    word_dicts = df['words'].to_numpy()
    total_dict = defaultdict(int)
    total_count = 0
    for word_dict in word_dicts:
        for word in word_dict.keys():
            total_dict[word] += word_dict[word]
            total_count += word_dict[word]
    unique_count = len(list(total_dict.keys()))
    if verbose: 
        print("Amount of unique words: {:,}".format(unique_count), 
             "\nTotal word count: {:,}".format(total_count))
    
    word_to_onehot_location = {}
    for i, word in enumerate(list(total_dict.keys())):
        word_to_onehot_location[word] = i
        
    new_col = []
    for i, row in enumerate(word_dicts):
        new_entry = np.zeros(unique_count, dtype=int)
        for word in row:
            new_entry[word_to_onehot_location[word]] = 1
        new_col.append(new_entry)
    if in_place:
        new_df = df
    else:
        new_df = df.copy()
    new_df['one-hot'] = new_col
    return new_df

def sentence_type(df, in_place=False):
    """
    adds an array to each row
    this array contains the frequency of each setence type
    arr[0] = . | arr[1] = ? | arr[2] = !
    each entry is total occurences / total punctuation count
    """
    new_col = []
    for transcript in df['transcript']:
        new_entry = np.zeros(3)
        p_count, q_count, e_count = transcript.count('.'), transcript.count('?'), transcript.count('!')
        total_sentences = p_count + q_count + e_count + 1
        new_entry[0] = (p_count + 1)/ total_sentences
        new_entry[1] = (q_count + 1)/ total_sentences
        new_entry[2] = (e_count + 1)/ total_sentences
        new_col.append(new_entry)
    if in_place:
        new_df = df
    else:
        new_df = df.copy()
    new_df['sentence-type'] = new_col
    return new_df


In [14]:
## TFIDF Calculation


# NOT USING RIGHT NOW 
def tfidf_value(transcript):
    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform([transcript])
    #list_of_words_tfidf = tfIdfVectorizer.get_feature_names_out()
    tfidf_value = tfIdf.toarray()[0]
    return tfidf_value

def tfidf_words(transcript):
    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform([transcript])
    try:
        list_of_words_tfidf = tfIdfVectorizer.get_feature_names_out()
    except Error:
        list_of_words_tfidf = tfIdfVectorizer.get_feature_names()
    tfidf_words = list_of_words_tfidf
    return tfidf_words

def tfidf_dict(transcript):
    tfIdfVectorizer = TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform([transcript])
    try:
        list_of_words_tfidf = tfIdfVectorizer.get_feature_names_out()
    except Error:
        list_of_words_tfidf = tfIdfVectorizer.get_feature_names()
    tfidf_value = tfIdf.toarray()[0]
    tfidf_dict = dict(zip(list_of_words_tfidf,tfidf_value))
    return tfidf_dict

def add_tfidf(df: pd.DataFrame, in_place=False):
    if in_place:
        df = df.copy()
    df['tfidf'] = df['transcript'].apply(tfidf_value)
    df['tfidf_word'] = df['transcript'].apply(tfidf_words)
    df['tfidf_dict'] = df['transcript'].apply(tfidf_dict)
    return df

In [15]:
# NOT USING RIGHT NOW 
def get_feature_engineered():
    data = raw.copy()
    data = correct_dataframe_topics_column(data, in_place=True)
    data = group_df_topics(data, in_place=True)
    data = add_words(data, in_place=True)
    data = one_hot_encode(data, in_place=True)
    data = sentence_type(data, in_place=True)
    data = add_tfidf(data, in_place=True)
    data = data.drop(columns = [
    "talk_id", "speaker_1", "recorded_date", 
    "event", "duration", "url", "comments", "about_speakers", 
    "available_lang", "all_speakers", "native_lang"])
    return data

In [16]:
def get_cleaned_data():
    data = raw.copy()
    data = correct_dataframe_topics_column(data, in_place=True)
    data = group_df_topics(data, in_place=True)
    data = add_words(data, in_place=True)
    data = sentence_type(data, in_place=True)
    data = data.drop(columns = [
    "talk_id", "speaker_1", "recorded_date", "published_date", 
    "event", "url", "about_speakers", 
    "available_lang", "all_speakers", "native_lang", "occupations", "related_talks", "description", "sentence-type"])
    return data

In [17]:
data_eda = get_cleaned_data()

/Users/sebastian/miniforge3/envs/ted-talks/lib/python3.11/site-packages/pandas/core/strings/object_array.py:158: FutureWarning:

Possible nested set at position 1



In [18]:
data_eda["grouped_topics"].value_counts()

stem            1445
culture          946
activism         662
business         445
history          356
not-relevant     151
Name: grouped_topics, dtype: int64

In [19]:
data_eda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4005 entries, 0 to 4004
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           4005 non-null   object 
 1   views           4005 non-null   int64  
 2   comments        3350 non-null   float64
 3   duration        4005 non-null   int64  
 4   topics          4005 non-null   object 
 5   transcript      4005 non-null   object 
 6   grouped_topics  4005 non-null   object 
 7   clean_scripts   4005 non-null   object 
 8   words           4005 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 281.7+ KB


tfidf --> clean_script 
ohe --> all of the words 
ohe --> grouped topics 
standardize --> duration 
standardize --> views
standardize --> comments

In [20]:
df_count = pd.Series(keywords).to_frame().reset_index().rename(columns = {"index": "category", 0:"count"})

df_count = df_count[df_count["count"] >= 50]
total_count = df_count.shape[0]

lst = []
for i in range(0, total_count, 39):
    df_cut = df_count.iloc[i: i+ 39]
    lst.append(df_cut)

In [21]:
# np.array(data_eda["clean_scripts"])

In [22]:
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

In [23]:
X = data_eda.fillna(0)
y = data_eda["grouped_topics"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25) 

preproc = ColumnTransformer(
    transformers = [
        ("tfidf", TfidfVectorizer(stop_words = "english"), "clean_scripts"), # only need to be taken in a series
        #('std_views', StandardScaler(), ["views"]), 
        #('std_comments', StandardScaler(), ["comments"])
    ],
    remainder = "drop"
)

# basic idea is to weight each category evenly so they have an even 
# weight*proportion=1 so the model can guess all categories
props = (y_train.value_counts()/len(y_train))
weights = {'activism':1/props['activism'], 
                 'business':1/props['business'], 
                 'culture':1/props['culture'], 
                 'history':1/props['history'], 
                 'not-relevant':1/props['not-relevant'], 
                 'stem':1/props['stem']}

nb_pipeline = Pipeline([
    ('preprocessor', preproc),
    ('classifier', LinearSVC(C = 5,
                            class_weight = weights))
])

pl_pipeline = Pipeline([ 
    ('preproc', preproc),
    ("dec_tree", DecisionTreeClassifier(max_depth = 8, min_samples_split = 8))
])

svc_pipeline = Pipeline([
    ('preproc', preproc),
    ('svc', SVC(gamma='auto', C=5))
])

sgd_pipeline = Pipeline([
    ('preproc', preproc),
    ('SGD', SGDClassifier(loss='log_loss', max_iter=5000))
])

def test_model(model):
    model.fit(X_train, y_train)
    print(f"Train accuracy: {accuracy_score(model.predict(X_train), y_train)}")
    print(f"Test accuracy: {accuracy_score(model.predict(X_test), y_test)}")
    return model

#model = test_model(sgd_pipeline)

## problems with current model:
guesses too many `stem`: (guessed about 0.45 when it is 0.36) +1.5x<br>
guesses too few `not-relavant`: (guessed 0.000998 of 0.037) -37x diff<br>
guesses too few `history`(2nd least common): (guessed 0.038 of 0.09) -3x diff<br>
these remain decently the same when model run multiple times<br>
<br>
i tried to change the weights based on these observations but accuracy will prob go down if you change them too much

In [25]:
def get_probas(input_str, model):
    str_arr = pd.Series(input_str).to_frame()
    str_arr = str_arr.rename(columns={0: "clean_scripts"})
    predictions = model.predict_proba(str_arr)
    return predictions, model.predict(str_arr)[0]

In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    data_eda[['clean_scripts', 'grouped_topics', 'views', 'comments', 'duration']], 
    data_eda['grouped_topics'],
    test_size=0.2
)
X_train, X_test = X_train.fillna(0), X_test.fillna(0)

In [86]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

def draw_graph(y_test, predict_test):
    df_draw = y_test.to_frame()
    df_draw = df_draw.rename(columns = {"grouped_topics":"actual_topic"})

    check_correct = y_test == predict_test 
    check_correct = check_correct.apply(lambda x: 1 if x is True else 0)

    df_draw["correct"] = check_correct
    df_draw = df_draw.groupby("actual_topic").mean().sort_values(by= "correct").reset_index()

    import plotly.express as px
    fig = px.bar(df_draw, 
                 x='actual_topic', 
                 y='correct', 
                 color = "actual_topic",
                 labels={'actual_topic':'Topic', "correct":"Percentage Correctness"}, 
                 title = "Model Correctness on Test Data", 
                 color_discrete_sequence= px.colors.qualitative.G10
                )
    fig.update_layout(yaxis_range=[0,1])
    return fig
    
logistic_regression = LogisticRegression()
classifier = OneVsRestClassifier(logistic_regression)

preproc = ColumnTransformer(
    transformers = [
        ("tfidf", TfidfVectorizer(stop_words = "english"), "clean_scripts"), # only need to be taken in a series
        ('std_views', StandardScaler(), ["views"]), 
        ('std_comments', StandardScaler(), ["comments"])
    ],
    remainder = "drop"
)

log_pipeline = Pipeline([
    ('preprocessor', preproc),
    ('classifier_log', classifier)
])

log_pipeline.fit(X_train, y_train)
predict = log_pipeline.predict(X_train)
print ( accuracy_score(y_train, predict)) 
print ("train")

predict_test_log = log_pipeline.predict(X_test)
print (accuracy_score(y_test, predict_test_log)) 
print ("test")

fig = draw_graph(y_test, predict_test_log)

0.7471910112359551
train
0.5468164794007491
test


In [36]:
log_pipeline.predict(pd.DataFrame.from_dict({'clean_scripts': ['hello there']}))
fig.show()

In [71]:
preproc = ColumnTransformer(
    transformers = [
        ("tfidf", TfidfVectorizer(stop_words = "english"), "clean_scripts"), # only need to be taken in a series
        ('std_views', StandardScaler(), ["views"]), 
        ('std_comments', StandardScaler(), ["duration"]),
    ],
    remainder = "drop"
)

ovr_classifier = Pipeline([
    ('preprocessor', preproc),
    ('classifier', OneVsRestClassifier(LinearSVC(C = 0.08,
                        multi_class ='ovr')))])

ovr_classifier.fit(X_train, y_train)
predict = ovr_classifier.predict(X_train)
accuracy_score(y_train, predict)
predict_test_ovr = ovr_classifier.predict(X_test)
accuracy_score(y_test, predict_test_ovr)
#fig = draw_graph(y_test, predict_test_ovr)

0.5493133583021224

In [69]:
X_train.shape, y_train.shape

((3204, 5), (3204,))

In [87]:
import plotly.graph_objects as go
train_accuracy = {}
test_accuracy = {}
model = log_pipeline
for topic in y_train.unique():
    if topic == 'not-relevant':
        continue
    topic_specific = X_train['grouped_topics'] == topic
    train_accuracy[topic] = model.score(X_train[topic_specific], y_train[topic_specific])
    topic_specific = X_test['grouped_topics'] == topic
    test_accuracy[topic] = model.score(X_test[topic_specific], y_test[topic_specific])

topics = list(train_accuracy.keys())
train_scores = list(train_accuracy.values())
test_scores = list(test_accuracy.values())

keys = list(train_accuracy.keys())
train_values = list(train_accuracy.values())
test_values = list(test_accuracy.values())
fig = go.Figure(data=[
    go.Bar(name='Train Accuracy', x=keys, y=train_values),
    go.Bar(name='Test Accuracy', x=keys, y=test_values)
])
fig.update_layout(title='Train Accuracy vs Test Accuracy',
                  xaxis_title='Topics',
                  yaxis_title='Accuracies',
                  barmode='group')
fig.show()

In [76]:
with open("./streamlit_resources/log_model_accuracy_2.json", 'w+') as f:
    f.write(fig.to_json())
